## Postprocessing randomized measurement on IBM quantum computers (

This tutorial illustrates how to acquire and save randomized measurements on a quantum computer using Qiskit (RandomizedMeasurementsQiskit.ipynb). This data can be then postprocessed using our julia library RandomMeas (RandomizedMeasurementsQiskitPostprocessing.ipynb)

In [27]:
using ITensors
using RandomMeas
using NPZ

### Loading the data and retrieving the parameters of the experiment

In [31]:
job_id = "bc323a46-8a83-47d6-af7d-51d7217dac4e"
u_ = npzread("data/"*job_id*"_u.npy")
data_cal = npzread("data/"*job_id*"_data.npy")[1,:,:,:]; #Data obtained from the circuit qc_meas used for calibrating robust shadows
#data_cal = 3 .*data_cal .- 1;
data = npzread("data/"*job_id*"_data.npy")[2,:,:,:]; #Data obtained from the preparation of the cluster state
Nu,NM,N = size(data)
@show Nu,NM,N;

(Nu, NM, N) = (512, 1024, 4)


### Converting the data into ITensor format 

In [32]:
ξ = siteinds("Qubit",N)
u = Vector{Vector{ITensor}}()
for r in 1:Nu
    ut = Vector{ITensor}()
    for i in 1:N
        push!(ut,ITensor(u_[r,i,:,:],ξ[i]',ξ[i]))
    end
    push!(u,ut)
end

### Extraction of the calibration parameters $G$ for building robust shadows

For purely readout error models, we have $G\approx 1-p$ where $p$ is the readout error 

In [33]:
G_e = zeros(Float64,N) #Estimations of G
states = ["Dn" for n in 1:N]
ψ0  = MPS(ComplexF64,ξ,states);
for r in 1:Nu
        ψu = apply(u[r],ψ0)
        for i in 1:N
            P = get_Born(data_cal[r,:,i:i],ξ[i:i]) #measured born probability on qubit i
            σui,_ = reduce_dm(ψu,i,i) #expected single qubit density matrix
            Pt = get_Born(σui) #expected born probability
            G_e[i] += real((3*(P-Pt)*Pt)[]+1)/Nu
        end
end
print("Calibrated RM parameters: ", G_e)

Calibrated RM parameters: [0.9914249272526252, 0.9948550003628972, 0.9972781597785446, 0.9734066921620793]

### Extracting the purity

We extract the purity of the first two qubits, and the purity of the four qubits. As expected for a cluster state, we have $p(12)\approx 0.5$, and $p(1234)\approx 1$.  

In [34]:
uA = [ut[1:2] for ut in u]
println("SubSystem Purity (unmitigated) ", get_purity_shadows(data[:,:,1:2],uA,ξ[1:2]))
println("SubSystem Purity (mitigated) ", get_purity_shadows(data[:,:,1:2],uA,ξ[1:2];G=G_e[1:2]))
println("Total Purity (unmitigatd) ", get_purity_shadows(data,u,ξ))
println("Total Purity (mitigated) ", get_purity_shadows(data,u,ξ;G=G_e))

SubSystem Purity (unmitigated) 0.47079739100643725
SubSystem Purity (mitigated) 0.4833479446519911
Total Purity (unmitigatd) 0.8473013797069321
Total Purity (mitigated) 0.96446404866567
